In [ ]:
"""
Bootstrap do ambiente do projeto.
Deve ser executada sempre como a primeira célula do notebook.
"""
import sys
from pathlib import Path

# Garante que a raiz do projeto esteja no PYTHONPATH
PROJECT_ROOT = Path().resolve().parents[0]
sys.path.append(str(PROJECT_ROOT))

from src.paths import create_project_structure
from src.config import set_global_seed

create_project_structure()
set_global_seed()


# Contexto do Problema

Os dados analisados neste projeto são provenientes de um processo industrial de flotação em uma mineradora.
A flotação é uma etapa crítica do processo produtivo, pois impacta diretamente a **qualidade do concentrado**
gerado e, consequentemente, o valor econômico do produto final.

Este projeto deve ser entendido como um **piloto exploratório de ciência de dados**, com os seguintes
aspectos relevantes:

- Conhecimento limitado do processo produtivo;
- Ausência de controle direto sobre a coleta dos dados;
- Cliente disposto a assumir risco técnico nesta etapa;
- Objetivo principal: avaliar se **há potencial preditivo** nos dados disponíveis.

Portanto, mais importante do que entregar um modelo altamente performático é **entender os dados,
suas limitações e suas possibilidades**.<br>
(O cliente quer saber o que é que dá pra fazer com esses dados)

# Flotação
Flotação é uma técnica de separação de misturas que consiste na introdução de bolhas de ar a uma suspensão de partículas.\
Com isso, verifica-se que as partículas aderem às bolhas, formando uma espuma que pode ser removida da solução, o que permite separar seus componentes de maneira efetiva.\
As partículas a serem flotadas são tornadas hidrofóbicas pela adição dos produtos químicos apropriados.\

## 1. O Processo de Flotação

Imagine que você tem um balde cheio de areia misturada com pequenos grãos de ouro (ou ferro, no caso desse dataset). Você quer separar o que é valioso do que é "sujeira" (sílica/areia). A flotação faz isso usando **bolhas de ar** e **química**.

1. **Moagem e Polpa:** O minério que sai da mina é triturado até virar um pó muito fino e misturado com água, formando uma "lama" chamada **polpa**.
2. **Condicionamento (A escolha do mineral):** Adicionamos reagentes químicos. No seu dataset, temos o **Starch (Amido)** e a **Amina**.
    * O **Amido** serve para "deprimir" o ferro (fazer com que ele queira ficar na água).
    * A **Amina** é o "coletor" da sílica (ela gruda nos grãos de areia e os torna "impermeáveis", ou hidrofóbicos).
3. **A Célula de Flotação:** Essa polpa vai para tanques (colunas de flotação). Lá, injetamos ar pelo fundo, criando milhares de microbolhas.
4. **O "Voo" dos Minerais:** Como a sílica está "impermeável" por causa da amina, ela não quer ficar na água. Ela prefere grudar nas bolhas de ar e subir.
5. **Coleta:** No topo da coluna, forma-se uma espuma rica em sílica (rejeito), que transborda e é descartada. O mineral "limpo" (o ferro), que ficou pesado e não grudou nas bolhas, sai pelo fundo do tanque. Isso é chamado de **flotação reversa**, que é muito comum no ferro: você flutua a impureza para limpar o produto.

## 2. Trade-off entre Teor de Ferro e Teor de Sílica

Em processos de beneficiamento mineral, especialmente na flotação, existe\
um **trade-off clássico** entre o teor de ferro e o teor de sílica no concentrado final.\
\
De forma simplificada:

- O objetivo do processo é **aumentar o teor de ferro** no concentrado;
- Ao mesmo tempo, busca-se **reduzir o teor de sílica**, que é considerada uma impureza.
Entretanto, essas duas metas **não são independentes**.\
Ajustes operacionais que favorecem a remoção de sílica\
podem, em determinados regimes, também afetar a recuperação de ferro,\
e vice-versa.

## 3. Origem do Trade-off no Processo de Flotação

A flotação é um processo físico-químico sensível a múltiplas variáveis, como:
- dosagem de reagentes (amina, amido);
- pH da polpa;
- densidade da polpa;
- vazão de ar e nível das colunas.

Essas variáveis influenciam simultaneamente:
- a seletividade do processo;
- a eficiência de separação entre minerais desejáveis e indesejáveis.

Como consequência, melhorar o desempenho em relação à sílica\
pode ocorrer às custas de pequenas perdas na recuperação de ferro,\
criando um compromisso operacional.

## 3. Implicações para Modelagem Preditiva

A presença desse trade-off implica que:
- As variáveis `% Iron Concentrate` e `% Silica Concentrate`\
  tendem a apresentar **correlação negativa**;
- Modelos treinados para prever apenas uma dessas variáveis\
  podem capturar efeitos indiretos sobre a outra;
- O problema pode ser encarado como:
  - dois problemas supervisionados independentes, ou
  - um problema multialvo, dependendo do objetivo analítico.

Reconhecer explicitamente esse trade-off ajuda a evitar interpretações\
equivocadas dos resultados e reforça a necessidade de análise cuidadosa.

## 4. Impacto na Tomada de Decisão

Do ponto de vista de negócio, o trade-off entre teor de ferro e teor de sílica\
reflete decisões reais do processo produtivo.
\
Em determinados cenários, pode ser aceitável:
- reduzir marginalmente o teor de ferro para obter uma redução significativa\
  no teor de sílica;
- ou aceitar um pouco mais de sílica em troca de maior recuperação de ferro.

Portanto, qualquer modelo preditivo deve ser interpretado à luz desse compromisso,\
e não apenas com base em métricas estatísticas isoladas.

## 5. Pré-Análise dos Dados

**O que é a curva Grade-Recovery?**

* **Grade (Teor):** É a pureza do seu produto (ex: % de Ferro no concentrado).
* **Recovery (Recuperação):** É a porcentagem de todo o ferro que entrou no processo e que você conseguiu realmente guardar no final.

Infelizmente, com as colunas disponíveis, **não conseguimos calcular a Recuperação Metalúrgica exata**. Para isso, precisaríamos de pelo menos uma dessas informações extras:

1. O teor de ferro no **Rejeito** (o que saiu na espuma).
2. O **fluxo de massa** (toneladas/hora) do concentrado final.

O dataset fornece o fluxo da polpa de alimentação (`Ore Pulp Flow`) e os teores de entrada e saída, mas sem o dado do rejeito, a conta não fecha matematicamente de forma precisa (usando a fórmula de duas correntes: ).

**Porém, há outras identidades importantes:**

1. **Relação Reagente/Alimentação:** É possível calcular quanto de Amina é necessário por tonelada de ferro que entra. Isso é crucial para o custo do processo.
2. **Qualidade do Concentrado:** O objetivo principal deste dataset é prever o `% Silica Concentrate`. Quanto menor a sílica, mais valioso é o ferro.
3. **Estabilidade das Colunas:** Há os níveis (`Level`) e fluxos de ar (`Air Flow`) de 7 colunas. Isso permite ver como a turbulência afeta a pureza final.

Pode-se criar uma **"Curva de Seletividade"**, plotando o `% Iron Concentrate` vs. `% Silica Concentrate`.\
Isso mostrará o quão eficiente o processo é em remover a sujeira mantendo o ferro alto.\
\
**Identidades Importantes no CSV:**
    **Amina Flow / Ore Pulp Flow:** É o seu "Dosage Rate". É a variável que os operadores mais mexem para controlar a qualidade.
    **Ore Pulp pH:** Na flotação de ferro, o pH deve ser mantido geralmente alcalino (perto de 10) para que os reagentes funcionem. A coluna `Ore Pulp pH` deve ser muito estável.
    **Colunas de Ar e Nível:** Se o nível da coluna sobe muito, a espuma sai mais rápido, levando mais ferro junto (aumenta recuperação, mas piora o teor).\
    Se o nível desce, a espuma fica mais "seca" e limpa (melhora o teor, mas você perde mais ferro no rejeito).

# Objetivos

O objetivo principal é usar esses dados para prever o nível de impureza no concentrado de minério. Como essa impureza é medida a cada hora,\
se conseguirmos prever quanta sílica (impureza) existe no concentrado, podemos ajudar os engenheiros, fornecendo informações antecipadas\
para que tomem decisões (empoderando-os!). Assim, eles serão capazes de realizar ações corretivas com antecedência (reduzir a impureza, se for\
o caso) e também ajudar o meio ambiente (reduzindo a quantidade de minério que vai para os rejeitos à medida que se reduz a sílica no concentrado).\

* É possível prever o % de Sílica no Concentrado a cada minuto?
* Com quantas etapas (horas) de antecedência conseguimos prever o % de Sílica no Concentrado? Isso ajudaria os engenheiros a agir de forma preditiva\
e otimizada, mitigando o % de ferro que poderia ter ido para os rejeitos.
* É possível prever o % de Sílica no Concentrado sem utilizar a coluna de % de Ferro no Concentrado (já que elas são altamente correlacionadas)?

# Dicionário de Dados

Aqui está o dicionário de dados formatado em Markdown, pronto para ser copiado para o seu relatório ou para a documentação (README) do seu projeto:

### 1. Identificação Temporal

| Coluna | Descrição | Unidade |
| --- | --- | --- |
| `date` | Data e hora do registro (timestamp). No dataset, os dados são registrados a cada 20 segundos. | Data/Hora |

### 2. Variáveis de Alimentação (Entrada)

Representam as características do minério bruto que chega da mina/moagem.
| Coluna | Descrição | Unidade |
| :--- | :--- | :--- |
| `% Iron Feed` | Percentual de Ferro no minério que está sendo alimentado nas células de flotação. | % |
| `% Silica Feed` | Percentual de Sílica (impureza) no minério que está sendo alimentado. | % |

### 3. Reagentes Químicos

Variáveis de controle utilizadas pelos engenheiros para otimizar a separação.
| Coluna | Descrição | Unidade |
| :--- | :--- | :--- |
| `Starch Flow` | Vazão de Amido. Atua como depressor, impedindo que o ferro flutue (hidrofilia). | m³/h |
| `Amina Flow` | Vazão de Amina. Atua como coletor, fazendo a sílica flutuar (hidrofobia).| m³/h |

### 4. Condições da Polpa

Variáveis físicas do meio em que ocorre a reação.
| Coluna | Descrição | Unidade |
| :--- | :--- | :--- |
| `Ore Pulp Flow` | Vazão mássica da polpa de minério. | t/h |
| `Ore Pulp pH` | Nível de pH da polpa (escala de 0 a 14). | pH |
| `Ore Pulp Density` | Densidade da polpa (relação massa/volume). | kg/cm³ |

### 5. Sensores das Colunas de Flotação (01 a 07)

Cada uma das 7 colunas possui sensores individuais de ar e nível.
| Coluna | Descrição | Unidade |
| :--- | :--- | :--- |
| `Flotation Column 0X Air Flow` | Fluxo de ar injetado na célula para criar as microbolhas. | Nm³/h |
| `Flotation Column 0X Level` | Altura da camada de espuma (froth level) dentro da célula. | mm |

### 6. Variáveis de Qualidade (Saída/Target)

Resultados finais do processo, geralmente medidos via análise laboratorial.
| Coluna | Descrição | Unidade |
| :--- | :--- | :--- |
| `% Iron Concentrate` | Teor final de Ferro no concentrado. Indica a riqueza do produto. | % |
| `% Silica Concentrate` | Teor final de Sílica no concentrado. Indica a pureza (Alvo de Predição). | % |

---

* **Variáveis Preditivas:** Seções 2, 3, 4 e 5.
* **Variável Alvo (Target):** `% Silica Concentrate`.
* **Relação de Dependência:** `% Iron Concentrate` e `% Silica Concentrate` são dependentes. Em um modelo preditivo realista, não se deve usar um para prever o outro,\
pois ambos são resultados do laboratório que chegam ao mesmo tempo.
* **Diferença de Frequência:** As colunas de "Processo (Sensor)" são geradas automaticamente pelo sistema (SCADA) em tempo real. As colunas de "Saída (Laboratório)" são\
inseridas manualmente a cada hora.

# Hipóteses Iniciais

Antes de explorar os dados, é importante explicitar hipóteses iniciais.
Essas hipóteses ajudam a guiar a análise de forma estruturada.

**Hipótese 1**  
Variáveis de processo medidas ao longo do tempo possuem relação estatística
com a qualidade final do concentrado.

**Hipótese 2**  
Existe defasagem temporal entre mudanças no processo e impacto na qualidade,
o que sugere que variáveis defasadas (*lags*) podem ser informativas.<br>
(Isto é, uma flutuação em uma variável pode ter efeito retardado sobre a qualidade, provavelmente o amina flow)

**Hipótese 3**  
O processo pode operar em diferentes regimes ao longo do tempo, afetando
a estabilidade das relações observadas.

**Hipótese 4**  
Nem todas as variáveis disponíveis são igualmente relevantes; pode haver
redundância ou variáveis com baixo poder informativo.

Essas hipóteses orientarão as decisões nas próximas etapas do projeto.


# O que se espera observar nos dados?

Com base nas hipóteses formuladas, espera-se observar:

- Correlações moderadas entre algumas variáveis de processo e a variável alvo;
- Padrões temporais consistentes em determinadas janelas;
- Possíveis mudanças de comportamento ao longo do tempo;
- Ganhos incrementais ao incorporar contexto temporal nas features.

Caso esses sinais não estejam presentes, a conclusão pode ser que,
com os dados atuais, **não há base suficiente** para justificar
modelagem preditiva avançada.


# Escopo do Projeto

**Dentro do escopo deste estudo:**
- Análise exploratória estruturada;
- Avaliação da qualidade dos dados;
- Uso de modelos simples e interpretáveis;
- Discussão de limitações e próximos passos.

**Fora do escopo:**
- Otimização pesada de hiperparâmetros;
- Modelos complexos (ex.: deep learning);
- Deploy em produção;
- Inferência causal (embora a inferência causal estatística esteja fora,\
a validação de correlações frente à teoria física está dentro).

Essa delimitação é fundamental para alinhar expectativas com o cliente.

# Próximos Passos

No próximo notebook, avançaremos para:
- entendimento detalhado da estrutura dos dados;
- análise de qualidade;
- identificação de problemas que possam inviabilizar ou limitar a modelagem.

Este notebook serve como a **fundação conceitual** de todo o projeto.